In [7]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import stream_ndarray_adapter_datastore as snads
from quantization import pq_quantizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


pq_quantizer.get_cluster_centers() returns subspaced clusters. E.g. product quantizing sifts results in
ndarray of shape (2, 300, 64), where 2-number of subspaces (n_quantizers=2), 300-number of clusters in each subspace, 
64 - size of splitted original vector (128/2).
Saving this array to sqlite through StreamNdarrayAdapterDataStore results in 2 rows with ndarray of shape (300, 64) in table.
Maybe more convenient and consistent way to store items in our id-item approach is to store 600 rows of ndarrays with shape (64), keeping in mind, that first 300 clusters are from first subspace and other 300 - from second subspace.

In [8]:
sqlite_db_path_in='ds_data\example_compute_siftid_sift_imgid_samples0.1-clusters'
sqlite_table_name_in='clusterid_cluster'
sqlite_db_path_out_reshaped='ds_data\example_compute_siftid_sift_imgid_samples0.1-clusters2'
sqlite_table_name_out_reshaped='clusterid_cluster'

In [9]:
ds_in = sqlds.SQLiteTableDataStore(sqlite_db_path_in, sqlite_table_name_in)
ds_out_reshaped = sqlds.SQLiteTableDataStore(sqlite_db_path_out_reshaped, sqlite_table_name_out_reshaped)

In [10]:
ds_out_reshaped = sqlds.SQLiteTableDataStore(sqlite_db_path_out_reshaped, sqlite_table_name_out_reshaped)

In [11]:
with ds_in, ds_out_reshaped:
    ds_out_reshaped_adapter=snads.StreamNdarrayAdapterDataStore(ds_out_reshaped, element_n_dims_save=1)
    ds_out_adapter=snads.StreamNdarrayAdapterDataStore(ds_in, detect_final_shape_by_first_elem=True)
    ds_out_reshaped_adapter.save_items_sorted_by_ids(ds_out_adapter.get_items_sorted_by_ids())    

In [12]:
with ds_out_reshaped:
    print(ds_out_reshaped.get_count())
    print(next(ds_out_reshaped.get_items_sorted_by_ids()).shape)

600
(64,)
